# Learning Particle Optimization

In [2]:
# Import modules
import numpy as np
# Import PySwarms
import pyswarms as ps

# Cost Function

In [3]:
# Python3 code to find sum of Manhattan 
# distances between all the pairs of 
# given points 

# Return the sum of distance of one axis. 
def distancesum (arr, n): 
	
	# sorting the array. 
	arr.sort() 
	
	# for each point, finding 
	# the distance. 
	res = 0
	sum = 0
	for i in range(n): 
		res += (arr[i] * i - sum) 
		sum += arr[i] 
	
	return res 
	
def totaldistancesum( x , y , n ): 
	return distancesum(x, n) + distancesum(y, n) 
'''
# Driven Code 
x = [ -1, 1, 3, 2 ] 
y = [ 5, 6, 5, 3 ] 
n = len(x) 
print(totaldistancesum(x, y, n) ) 
'''
# This code is contributed by "Sharad_Bhardwaj". 


'\n# Driven Code \nx = [ -1, 1, 3, 2 ] \ny = [ 5, 6, 5, 3 ] \nn = len(x) \nprint(totaldistancesum(x, y, n) ) \n'

In [4]:
# -*- coding: utf-8 -*-
"""
Created on: xxxx
@author   : Anon

NAME
    Farm_Evalautor.py
    
PYTHON VERSION   
    3.7.3 
    
DESCRIPTION
    Calculates Annual Energy Production (AEP) of a Wind Farm
    ============================================================    
    
    This is vectorzied version of Farm_Evalautor.py. 
    Farm_Evalautor_Vec.py is a python file that calculates AEP (GWh)
    of a certain arrangement of wind turbines in a farm, under 
    given annual wind conditions. 
    
    The code in this script for wake-effect modeling is based on
    standard Jensen (PARK) model. 
    I. Katic, J. Hojstrup and N. Jensen, "A simple model for cluster 
    efficiency," in European Wind Energy Association Conference and 
    Exhibition, 1986.
    
    As its inputs, the code takes three data files containing info 
    about:
    - Turbine Locations
    - Turbine Power Curve
    - Annual Wind Conditions
    
PACKAGE LIST
    You may need to install the package Shapely in your
    python distribution. These are not pre-installed. 
    =============================================================
    Packages Used:
    Numpy
    Pandas
    Shapely
    math (built-in)
    
OPTIMIZATION USAGE
    This vectorized version is faster than unvectorized version
    Farm_Evalautor.py. Due to speed benefits, we advise you to use 
    the function getAEP in this script while developing the optimizer. 
    =============================================================
    
    One way to use getAEP function as AEP evaluator while optimizing is:
    - STEP 1. Import the relevant function from Farm_Evalautor_Vec. 
              from Farm_Evalautor_Vec import getTurbLoc, loadPowerCurve, 
              binWindResourceData, preProcessing, getAEP
    - STEP 2. Set Turbine Radius to 50.0. First arg of getAEP
    - STEP 3. Load Turbine Locations. Using function getTurbLoc
    - STEP 4. Load Power Curve. Using function loadPowerCurve
    - STEP 5. Load wind instance probabilities. 
              Using function binWindResourceData
    - STEP 6. Perform Preprocessing by calling function preProcessing.
              We do preprocessing to avoid same repeating calculations.
              Do them once. 
    - STEP 7. Finally, call function getAEP
    
    This makes it easy to control the reloading of data and hence achieve
    better performance.      
"""

# Module List
import numpy  as np
import pandas as pd                     
from   math   import radians as DegToRad       # Degrees to radians Conversion

from shapely.geometry import Point             # Imported for constraint checking
from shapely.geometry.polygon import Polygon

import warnings
warnings.filterwarnings("ignore")

def getTurbLoc(turb_loc_file_name):
    """ 
    -**-THIS FUNCTION SHOULD NOT BE MODIFIED-**-
    
    Returns x,y turbine coordinates
    
    :Called from
        main function
    
    :param
        turb_loc_file_name - Turbine Loc csv file location
        
    :return
        2D array
    """
    
    df = pd.read_csv(turb_loc_file_name, sep=',', dtype = np.float32)
    turb_coords = df.to_numpy(dtype = np.float32)
    return(turb_coords)


def loadPowerCurve(power_curve_file_name):
    """
    -**-THIS FUNCTION SHOULD NOT BE MODIFIED-**-
    
    Returns a 2D numpy array with information about
    turbine thrust coeffecient and power curve of the 
    turbine for given wind speed
    
    :called_from
        main function
    
    :param
        power_curve_file_name - power curve csv file location
        
    :return
        Returns a 2D numpy array with cols Wind Speed (m/s), 
        Thrust Coeffecient (non dimensional), Power (MW)
    """
    powerCurve = pd.read_csv(power_curve_file_name, sep=',', dtype = np.float32)
    powerCurve = powerCurve.to_numpy(dtype = np.float32)
    return(powerCurve)
    

def binWindResourceData(wind_data_file_name):
    r"""
    -**-THIS FUNCTION SHOULD NOT BE MODIFIED-**-
    
    Loads the wind data. Returns a 2D array with shape (36,15). 
    Each cell in  array is a wind direction and speed 'instance'. 
    Values in a cell correspond to probability of instance
    occurence.  
    
    :Called from
        main function
        
    :param
        wind_data_file_name - Wind Resource csv file  
        
    :return
        1-D flattened array of the 2-D array shown below. Values 
        inside cells, rough probabilities of wind instance occurence. 
        Along: Row-direction (drct), Column-Speed (s). Array flattened
        for vectorization purpose. 
        
                      |0<=s<2|2<=s<4| ...  |26<=s<28|28<=s<30|
        |_____________|______|______|______|________|________|
        | drct = 360  |  --  |  --  |  --  |   --   |   --   |
        | drct = 10   |  --  |  --  |  --  |   --   |   --   |
        | drct = 20   |  --  |  --  |  --  |   --   |   --   |
        |   ....      |  --  |  --  |  --  |   --   |   --   |
        | drct = 340  |  --  |  --  |  --  |   --   |   --   |
        | drct = 350  |  --  |  --  |  --  |   --   |   --   |        
    """
    
    # Load wind data. Then, extracts the 'drct', 'sped' columns
    df = pd.read_csv(wind_data_file_name)
    wind_resource = df[['drct', 'sped']].to_numpy(dtype = np.float32)
    
    # direction 'slices' in degrees
    slices_drct   = np.roll(np.arange(10, 361, 10, dtype=np.float32), 1)
    ## slices_drct   = [360, 10.0, 20.0.......340, 350]
    n_slices_drct = slices_drct.shape[0]
    
    # speed 'slices'
    slices_sped   = [0.0, 2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 14.0, 16.0, 
                        18.0, 20.0, 22.0, 24.0, 26.0, 28.0, 30.0]
    n_slices_sped = len(slices_sped)-1

    
    # placeholder for binned wind
    binned_wind = np.zeros((n_slices_drct, n_slices_sped), 
                           dtype = np.float32)
    
    # 'trap' data points inside the bins. 
    for i in range(n_slices_drct):
        for j in range(n_slices_sped):     
            
            # because we already have drct in the multiples of 10
            foo = wind_resource[(wind_resource[:,0] == slices_drct[i])] 

            foo = foo[(foo[:,1] >= slices_sped[j]) 
                          & (foo[:,1] <  slices_sped[j+1])]
            
            binned_wind[i,j] = foo.shape[0]  
    
    wind_inst_freq   = binned_wind/np.sum(binned_wind)
    wind_inst_freq   = wind_inst_freq.ravel()
    
    return(wind_inst_freq)


def searchSorted(lookup, sample_array):
    """
    -**-THIS FUNCTION SHOULD NOT BE MODIFIED-**-
    
    Returns lookup indices for closest values w.r.t sample_array elements
    
    :called_from
        preProcessing, getAEP
    
    :param
        lookup       - The lookup array
        sample_array - Array, whose elements need to be matched
                       against lookup elements. 
        
    :return
        lookup indices for closest values w.r.t sample_array elements 
    """
    lookup_middles = lookup[1:] - np.diff(lookup.astype('f'))/2
    idx1 = np.searchsorted(lookup_middles, sample_array)
    indices = np.arange(lookup.shape[0])[idx1]
    return indices

   

def preProcessing(power_curve):
    """
    -**-THIS FUNCTION SHOULD NOT BE MODIFIED-**-
    
    Doing preprocessing to avoid the same repeating calculations.
    Record the required data for calculations. Do that once.
    Data are set up (shaped) to assist vectorization. Used later in
    function totalAEP. 
    
    :called_from
        main function
    
    :param
        power_curve - 2D numpy array with cols Wind Speed (m/s), 
                      Thrust Coeffecient (non dimensional), Power (MW)
        
    :return
        n_wind_instances  - number of wind instances (int)
        cos_dir           - For coordinate transformation 
                            2D Array. Shape (n_wind_instances,1)
        sin_dir           - For coordinate transformation 
                            2D Array. Shape (n_wind_instances,1)
        wind_sped_stacked - column staked all speed instances n_turb times. 
        C_t               - 3D array with shape (n_wind_instances, n_turbs, n_turbs)
                            Value changing only along axis=0. C_t, thrust coeff.
                            values for all speed instances. 
    """
    # number of turbines
    n_turbs       =   50
    
    # direction 'slices' in degrees
    slices_drct   = np.roll(np.arange(10, 361, 10, dtype=np.float32), 1)
    ## slices_drct   = [360, 10.0, 20.0.......340, 350]
    n_slices_drct = slices_drct.shape[0]
    
    # speed 'slices'
    slices_sped   = [0.0, 2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 14.0, 16.0, 
                        18.0, 20.0, 22.0, 24.0, 26.0, 28.0, 30.0]
    n_slices_sped = len(slices_sped)-1
    
    # number of wind instances
    n_wind_instances = (n_slices_drct)*(n_slices_sped)
    
    # Create wind instances. There are two columns in the wind instance array
    # First Column - Wind Speed. Second Column - Wind Direction
    # Shape of wind_instances (n_wind_instances,2). 
    # Values [1.,360.],[3.,360.],[5.,360.]...[25.,350.],[27.,350.],29.,350.]
    wind_instances = np.zeros((n_wind_instances,2), dtype=np.float32)
    counter = 0
    for i in range(n_slices_drct):
        for j in range(n_slices_sped): 
            
            wind_drct =  slices_drct[i]
            wind_sped = (slices_sped[j] + slices_sped[j+1])/2
            
            wind_instances[counter,0] = wind_sped
            wind_instances[counter,1] = wind_drct
            counter += 1

	# So that the wind flow direction aligns with the +ve x-axis.			
    # Convert inflow wind direction from degrees to radians
    wind_drcts =  np.radians(wind_instances[:,1] - 90)
    # For coordinate transformation 
    cos_dir = np.cos(wind_drcts).reshape(n_wind_instances,1)
    sin_dir = np.sin(wind_drcts).reshape(n_wind_instances,1)
    
    # create copies of n_wind_instances wind speeds from wind_instances
    wind_sped_stacked = np.column_stack([wind_instances[:,0]]*n_turbs)
   
    # Pre-prepare matrix with stored thrust coeffecient C_t values for 
    # n_wind_instances shape (n_wind_instances, n_turbs, n_turbs). 
    # Value changing only along axis=0. C_t, thrust coeff. values for all 
    # speed instances.
    # we use power_curve data as look up to estimate the thrust coeff.
    # of the turbine for the corresponding closest matching wind speed
    indices = searchSorted(power_curve[:,0], wind_instances[:,0])
    C_t     = power_curve[indices,1]
    # stacking and reshaping to assist vectorization
    C_t     = np.column_stack([C_t]*(n_turbs*n_turbs))
    C_t     = C_t.reshape(n_wind_instances, n_turbs, n_turbs)
    
    return(n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t)


def getAEP(turb_coords):#turb_rad, turb_coords, power_curve, wind_inst_freq, 
            #n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t):
    coordinates=[]
    coordinates_particles=[]
    for j in range(50):
        coordinates=[]
        for i in range(0,100,2):
            coordinates.append([turb_coords[j][i],turb_coords[j][i+1]])
        coordinates_particles.append(coordinates)
    coordinates_particles=np.asarray(coordinates_particles)
#################################################################################333
    turb_specs    =  {   
                         'Name': 'Anon Name',
                         'Vendor': 'Anon Vendor',
                         'Type': 'Anon Type',
                         'Dia (m)': 100,
                         'Rotor Area (m2)': 7853,
                         'Hub Height (m)': 100,
                         'Cut-in Wind Speed (m/s)': 3.5,
                         'Cut-out Wind Speed (m/s)': 25,
                         'Rated Wind Speed (m/s)': 15,
                         'Rated Power (MW)': 3
                     }
    turb_diam      =  turb_specs['Dia (m)']
    turb_rad       =  turb_diam/2 
    
    # Turbine x,y coordinatesfrom pyswarms.utils.plotters.formatters import Mesher
    ##turb_coords   =  getTurbLoc(r'turbine_loc_test.csv')
    
    # Load the power curve
    power_curve   =  loadPowerCurve('power_curve.csv')
    
    # Pass wind data csv file location to function binWindResourceData.
    # Retrieve probabilities of wind instance occurence.
    wind_inst_freq =  binWindResourceData(r'wind_data_2007.csv')   
    
    # Doing preprocessing to avoid the same repeating calculations. Record 
    # the required data for calculations. Do that once. Data are set up (shaped)
    # to assist vectorization. Used later in function totalAEP.
    n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t = preProcessing(power_curve)
####################################################################    
    """
    -**-THIS FUNCTION SHOULD NOT BE MODIFIED-**-
    
    Calculates AEP of the wind farm. Vectorised version.
    
    :called from
        main
        
    :param
        turb_diam         - Radius of the turbine (m)
        turb_coords       - 2D array turbine euclidean x,y coordinates
        power_curve       - For estimating power. 
        wind_inst_freq    - 1-D flattened with rough probabilities of 
                            wind instance occurence.
                            n_wind_instances  - number of wind instances (int)
        cos_dir           - For coordinate transformation 
                            2D Array. Shape (n_wind_instances,1)
        sin_dir           - For coordinate transformation 
                            2D Array. Shape (n_wind_instances,1)
        wind_sped_stacked - column staked all speed instances n_turb times. 
        C_t               - 3D array with shape (n_wind_instances, n_turbs, n_turbs)
                            Value changing only along axis=0. C_t, thrust coeff.
                            values for all speed instances. 
    
    :return
        wind farm AEP in Gigawatt Hours, GWh (float)
    """
    cost=[]
    
    for i in coordinates_particles:
        #print(len(i))
        jasper=np.transpose(i)
        t=turb_coords=i
    # number of turbines
        n_turbs        =   turb_coords.shape[0]
        assert n_turbs ==  50, "Error! Number of turbines is not 50."

        # Prepare the rotated coordinates wrt the wind direction i.e downwind(x) & crosswind(y) 
        # coordinates wrt to the wind direction for each direction in wind_instances array
        rotate_coords   =  np.zeros((n_wind_instances, n_turbs, 2), dtype=np.float32)
        # Coordinate Transformation. Rotate coordinates to downwind, crosswind coordinates
        rotate_coords[:,:,0] =  np.matmul(cos_dir, np.transpose(turb_coords[:,0].reshape(n_turbs,1))) - \
                               np.matmul(sin_dir, np.transpose(turb_coords[:,1].reshape(n_turbs,1)))
        rotate_coords[:,:,1] =  np.matmul(sin_dir, np.transpose(turb_coords[:,0].reshape(n_turbs,1))) +\
                               np.matmul(cos_dir, np.transpose(turb_coords[:,1].reshape(n_turbs,1)))


        # x_dist - x dist between turbine pairs wrt downwind/crosswind coordinates)
        # for each wind instance
        x_dist = np.zeros((n_wind_instances,n_turbs,n_turbs), dtype=np.float32)
        for i in range(n_wind_instances):
            tmp = rotate_coords[i,:,0].repeat(n_turbs).reshape(n_turbs, n_turbs)
            x_dist[i] = tmp - tmp.transpose()


        # y_dist - y dist between turbine pairs wrt downwind/crosswind coordinates)
        # for each wind instance    
        y_dist = np.zeros((n_wind_instances,n_turbs,n_turbs), dtype=np.float32)
        for i in range(n_wind_instances):
            tmp = rotate_coords[i,:,1].repeat(n_turbs).reshape(n_turbs, n_turbs)
            y_dist[i] = tmp - tmp.transpose()
        y_dist = np.abs(y_dist) 


        # Now use element wise operations to calculate speed deficit.
        # kw, wake decay constant presetted to 0.05
        # use the jensen's model formula. 
        # no wake effect of turbine on itself. either j not an upstream or wake 
        # not happening on i because its outside of the wake region of j
        # For some values of x_dist here RuntimeWarning: divide by zero may occur
        # That occurs for negative x_dist. Those we anyway mark as zeros. 
        sped_deficit = (1-np.sqrt(1-C_t))*((turb_rad/(turb_rad + 0.05*x_dist))**2) 
        sped_deficit[((x_dist <= 0) | ((x_dist > 0) & (y_dist > (turb_rad + 0.05*x_dist))))] = 0.0


        # Calculate Total speed deficit from all upstream turbs, using sqrt of sum of sqrs
        sped_deficit_eff  = np.sqrt(np.sum(np.square(sped_deficit), axis = 2))


        # Element wise multiply the above with (1- sped_deficit_eff) to get
        # effective windspeed due to the happening wake
        wind_sped_eff     = wind_sped_stacked*(1.0-sped_deficit_eff)


        # Estimate power from power_curve look up for wind_sped_eff
        indices = searchSorted(power_curve[:,0], wind_sped_eff.ravel())
        power   = power_curve[indices,2]
        power   = power.reshape(n_wind_instances,n_turbs)

        # Farm power for single wind instance 
        power   = np.sum(power, axis=1)

        # multiply the respective values with the wind instance probabilities 
        # year_hours = 8760.0
        AEP = 8760.0*np.sum(power*wind_inst_freq)

        # Convert MWh to GWh
        AEP = AEP/1e3
        cost.append(-AEP-totaldistancesum(jasper[0], jasper[1], len(jasper[0])))
    return(cost)
    

    


# Optimizer Function

In [9]:
def cost_function(x):
    print(len(x))
    return x.sum()
x_max = 3950 * np.ones(100)
x_min = 50 * np.ones(100)
bounds = (x_min, x_max)
#bounds

In [10]:
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}
#turb_coords   =  getTurbLoc(r'turbine_loc_test.csv')
optimizer = ps.single.GlobalBestPSO(n_particles=50, dimensions=100, options=options,bounds = (x_min, x_max),init_pos=np.asarray([caleb]*50))

# Start Optimization

In [11]:
cost, pos = optimizer.optimize(getAEP, iters=50000)#,kwargs={})

2020-09-30 21:37:11,101 - pyswarms.single.global_best - INFO - Optimize for 50000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|50000/50000, best_cost=-2.86e+6
2020-10-01 20:17:57,308 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -2856134.3934602356, best pos: [ 665.9102   803.4095  2430.4604  2027.8027  1188.3585  2345.9167
 2256.8755  1716.8341  2250.582   3949.999   1017.6831  2643.8906
 2252.0537  1720.4246  1191.2251  2339.5273  3312.6309  1725.5068
 1555.1681  3559.5881  3132.6238   783.986    667.09436  801.86743
 3503.866   2636.5405  2609.5808   504.46335  491.52008 1724.3265
 3153.9468  2642.7002  2082.213   2633.7559  2005.6324  1998.2802
 3141.89    3283.1738   486.4337  2938.0442   640.8381  1431.4202
 3314.7874  1114.4419   672.95245  806.90485 2090.9934   199.27576
 3513.0579  2628.6826  1892.9861  1100.5488  1198.4952  2336.2336
 3128.567   3260.3943  2607.0903   502.97278 2614.9656  3563.6794
  836.

In [12]:
print(cost)

-2856134.3934602356


In [13]:
coordinates=[]
for i in range(0,100,2):
    coordinates.append([pos[i],pos[i+1]])
coordinates    

[[665.9102, 803.4095],
 [2430.4604, 2027.8027],
 [1188.3585, 2345.9167],
 [2256.8755, 1716.8341],
 [2250.582, 3949.999],
 [1017.6831, 2643.8906],
 [2252.0537, 1720.4246],
 [1191.2251, 2339.5273],
 [3312.6309, 1725.5068],
 [1555.1681, 3559.5881],
 [3132.6238, 783.986],
 [667.09436, 801.86743],
 [3503.866, 2636.5405],
 [2609.5808, 504.46335],
 [491.52008, 1724.3265],
 [3153.9468, 2642.7002],
 [2082.213, 2633.7559],
 [2005.6324, 1998.2802],
 [3141.89, 3283.1738],
 [486.4337, 2938.0442],
 [640.8381, 1431.4202],
 [3314.7874, 1114.4419],
 [672.95245, 806.90485],
 [2090.9934, 199.27576],
 [3513.0579, 2628.6826],
 [1892.9861, 1100.5488],
 [1198.4952, 2336.2336],
 [3128.567, 3260.3943],
 [2607.0903, 502.97278],
 [2614.9656, 3563.6794],
 [836.8301, 1109.5973],
 [1020.14044, 804.18176],
 [3310.6963, 1113.7642],
 [3478.7039, 2662.5522],
 [1535.87, 1717.4219],
 [1912.9128, 1139.2545],
 [2770.5713, 1415.1865],
 [1375.5736, 3276.175],
 [3499.6267, 2648.6094],
 [311.1319, 2045.6466],
 [2424.265, 3257.

In [14]:
import numpy
a = coordinates 
numpy.savetxt("Result1323.csv", a, delimiter=",")


In [6]:
turb_coords   =  getTurbLoc(r'ResultBest.csv')

In [7]:
t=np.transpose(turb_coords)
len(t[0])

50

In [8]:
tur=[]
caleb=[]
for kal in range(len(t[0])):
    #caleb=np.zeros(kal*2).tolist()
    caleb.append(t[0][kal])
    caleb.append(t[1][kal])
    #caleb=caleb+np.zeros(np.abs(len(t[0])-i-1)*2).tolist()
    #tur.append(caleb)

In [170]:
len(caleb)

100